In [26]:
import pandas as pd
import numpy as np
import yfinance as yf
import re
from fuzzywuzzy import fuzz

In [27]:
def process_df(x):
    columns = x.iloc[0,:]
    columns.name = ""
    x.columns = columns
    x = x.iloc[1:,:]
    x.reset_index(drop=True, names=columns, inplace=True)
    x.rename(index={"6":""}).head(7)
    return x


def get_working_capital(zone=""):
    path = "data_damodaran/cash_flow_estimation/working_capital/wcdata" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_growth_historical(zone=""):
    path = "data_damodaran/growth/historical_growth/histgr" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_capex(zone=""):
    path = "data_damodaran/cash_flow_estimation/CAPEX/capex" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x


def get_taxrates(zone=""):
    path = "data_damodaran/discount_rate_estimation/tax/tax_rate_country/taxrate" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry name")
    return x


def get_growth_ebit(zone=""):
    path = "data_damodaran/growth/growth_ebit/fundgrEB" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x




def get_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_damodaran/totalbeta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_unleveraged_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_leverage/beta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_taxrates(zone=""):
    path = "data_damodaran/discount_rate_estimation/tax/tax_rate_country/taxrate" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry name")
    return x

def get_wacc(zone=""):
    path = "data_damodaran/discount_rate_estimation/wacc_damodaran/wacc" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_spread_classifier(size=5e9, financial=False):
    path = "data_damodaran/capital_structure/spread_classifiers/ratings.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Start here Ratings sheet"]
    moody = process_df(x.iloc[16:32,:4])
    moody_financefirms = process_df(x.iloc[16:,5:9])
    moody_smallfirms = process_df(x.iloc[35:,:4])
    
    if financial == True:
        return moody_financefirms
    else:
        if size >= 5e9:
            return moody
        else: 
            return moody_smallfirms
        
        
def get_10y_tbills():
    path = "data_damodaran/discount_rate_estimation/treasury_bills/histretSP.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["T. Bond yield & return"]
    x = x.iloc[5:,:]
    x = process_df(x)
    return x

def get_moody_spread(x, size=5e9, financial=False):
    for i in range(len(get_spread_classifier(size,financial=financial))):
        if x <= get_spread_classifier(size,financial=financial).iloc[i,1]:
            return get_spread_classifier(size,financial=financial).iloc[i,-1]
        else:
            continue
    return get_spread_classifier(size,financial=financial).iloc[-1,-1]


def get_risk_premium():
    path = "data_damodaran/discount_rate_estimation/risk_premium/ctrypremJuly23.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["ERPs by country"]
    x = x.iloc[5:163,:9]
    x = process_df(x)
    x = x.set_index("Country")
    return x

def get_gdp_growth(ticket, n):
    path = "Database/GDP by Country 1999-2022.csv"
    x = pd.read_csv(path)#, sheet_name=None)
    x.dropna(inplace=True)
    x = x.T
    x.columns = x.iloc[0,:]
    x = x.drop("Country")
    for i in x:
        x[i] = x[i].apply(lambda x: x.replace(",",""))
    x = x.astype("float")
    x = x.apply(lambda x: x.pct_change())
    x = x.iloc[-n:,:]
    x = np.exp(np.log(x).mean())  # We use geometric mean to deal with outliers
    
    country = yf.Ticker(ticket).info["country"]
    
    return x.loc[country]

def get_industry(ticket):
    ls = []
    ind = yf.Ticker(ticket).info.get("industry")
    for i in industries:
        ls.append([fuzz.ratio(ind, i),i])
    return pd.DataFrame(ls).sort_values(0, ascending=False).iloc[0,1]

sp500_list = pd.read_csv("Database/SP500.csv")["Symbol"]
industries = list(get_betas().index)


## Free Cash Flow

In [69]:
def freecashflow(ticket, sector,n=4):
    x = yf.Ticker(ticket)
    ebit = x.income_stmt.loc["EBIT",:].sort_index()
    g_ebit = get_growth_ebit("").loc[sector,"Expected Growth in EBIT"]
    if ebit.iloc[-1] < 0:
        ebit_forecast = pd.Series([ebit.iloc[-n:].mean()*((1+g_ebit)**i) for i in range(n+1)], index=range(0,n+1))[1:]
    else:
        ebit_forecast = pd.Series([ebit.iloc[-1]*((1+g_ebit)**i) for i in range(n+1)], index=range(0,n+1))[1:]
    tax_rate = get_taxrates("").loc[sector, "Average across only money-making companies"]

    usa_ktn_percentage = get_working_capital("").loc[sector,"Non-cash WC/ Sales"]
    sales_growth_2y = get_growth_historical("").loc[sector,"Expected Growth in Revenues - Next 2 years"]
    g_sales = (1+sales_growth_2y)**(1/2)-1

    sales = x.incomestmt.loc["Total Revenue",:].sort_index()
    sales_forecast = pd.Series([sales.iloc[-1]*((1+g_sales)**i) for i in range(n+1)], index=range(0,n+1))
    sales_forecast

    ktn_forecast = sales_forecast * usa_ktn_percentage
    ktn_delta_forecast = ktn_forecast.diff()[1:]

    capex_vs_sales = get_capex("").loc[sector, "Net Cap Ex/Sales"]
    capex_net = (sales_forecast * capex_vs_sales)[1:]


    return ebit_forecast*(1-tax_rate) - capex_net - ktn_delta_forecast


freecashflow("AMZN", "Retail (Online)",5)

1   -1.744706e+10
2   -9.440557e+09
3    1.955566e+09
4    1.798344e+10
5    4.033465e+10
dtype: float64

## WACC

In [50]:
def wacc(ticket,sector):
    x = yf.Ticker(ticket)
    E = x.balancesheet.loc["Stockholders Equity",:].iloc[0]
    D = x.balancesheet.loc["Total Liabilities Net Minority Interest",:].iloc[0]
    
    rf = get_10y_tbills().iloc[-1,1]
    rd = get_wacc().loc[sector,"Cost of Debt"]
    
    interest = x.incomestmt.loc["Interest Expense",:]
    ebit = x.incomestmt.loc["EBIT",:]
    coverage_ratio = ebit / interest
    spread = get_moody_spread(coverage_ratio.iloc[-1])
    rd = rf + spread
    
    tax_rate = get_taxrates("").loc[sector, "Average across only money-making companies"]
    beta_u = get_unleveraged_betas().loc[sector,"Unlevered beta"]
    beta = beta_u * (1 + (1-tax_rate)*D/E)
    prm = get_risk_premium().loc[x.info["country"],"Total Equity Risk Premium"]
    re = rf + beta * prm
    
    wacc = rd*(1-tax_rate)*(D/(E+D)) + re*(E/(D+E))
    return wacc

wacc("MSFT", "Software (System & Application)")


/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


0.10096140142386192

In [51]:
yf.Ticker("MSFT").info["country"]

'United States'

In [52]:
def EV(ticket,sector, n=4):
    dcf = []
    fcf_ls = freecashflow(ticket,sector,n)
    wacc_ = wacc(ticket,sector)
    g = get_gdp_growth(ticket,n)    

    for i in range(1,n+1):
        dcf.append(fcf_ls[i]/((1 + wacc_)**i))

    VT = (fcf_ls.iloc[-1]*(1-g))/(wacc_ - g)

    EV = sum(dcf) + VT
    return EV

In [145]:
x = [EV("MSFT","Software (System & Application)"), EV("AAPL","Software (System & Application)"),
    EV("NVDA","Semiconductor"), EV("DIS","Entertainment"), EV("KO","Beverage (Soft)")]

x

/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/leonplaza/miniconda3/lib/python3.11/site-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/leonplaza/miniconda3/lib/python3.11/site-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
/Users/leonplaza/miniconda3/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/Users/leonplaza/miniconda3/lib/python3.11/site-packages/pandas/core/internals/blocks.py:366: Ru

[2714873172289.39,
 2976423297373.0835,
 41160733840.12602,
 60220302728.93921,
 280759038566.04126]

In [134]:
y = [enterpriseValue("MSFT"),enterpriseValue("AAPL"),enterpriseValue("NVDA"),enterpriseValue("DIS"),
     enterpriseValue("KO")]



In [146]:
df = pd.concat([pd.Series(x),pd.Series(y)], axis=1)

#

df.index = ["Microsoft", "Apple","Nvidia","Disney","Coca Cola Company"]
df.rename({0:"Damodaran's Prediction",1:"Yahoo Finance's Prediction"}, axis=1)

df["% change"] = (df.iloc[:,0] / df.iloc[:,1] - 1) * 100
df.rename({0:"Damodaran's Prediction",1:"Yahoo Finance's Prediction"}, axis=1)

,Damodaran's Prediction,Yahoo Finance's Prediction,% change
Microsoft,2.714873e+12,2730402250752,-0.568747
Apple,2.976423e+12,3070588354560,-3.066678
Nvidia,4.116073e+10,1142926213120,-96.398653
Disney,6.022030e+10,216399380480,-72.171684
Coca Cola Company,2.807590e+11,281527451648,-0.272944


In [54]:
def enterpriseValue(ticket):
    x = yf.Ticker(ticket)
    return x.info.get("enterpriseValue")

In [ ]:
smpl = sp500_list.sample(5)

print(EV("MSFT","Software (System & Application)"), enterpriseValue("MSFT"))
print(EV("AAPL","Software (System & Application)"), enterpriseValue("AAPL"))
print(EV("AMZN","Retail (Online)"), enterpriseValue("AMZN"))